In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tqdm
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from dca_research.lqg import LQGComponentsAnalysis as LQGCA

/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from matplotlib.patches import Ellipse

In [ ]:
# Schematic Figure - 2D example of how non-normality affecs PCA/DCA divergence
l1 = 0.9
l2 = 0.8
phi = np.linspace(0, 1, 5)

pca_dca_angle = np.zeros(phi.size)

pcacoef = np.zeros((phi.size, 2, 2))
dcacoef = np.zeros((phi.size, 2, 2))

sscoef = np.zeros((phi.size, 2, 2))

sseig = np.zeros((phi.size, 2))
variance = np.zeros((phi.size, 2))
pi = np.zeros((phi.size, 2))

for i, phi_ in tqdm(enumerate(phi)):
    A = np.array([[l1, phi_], [0, l2]])
    
    ssm = SSR(A = A, B=np.eye(2), C=np.eye(2))
    eig, U = np.linalg.eig(ssm.P)
    sseig[i, :] = eig
    sscoef[i, ...] = U
    
    y = ssm.trajectory(int(1e5), burnoff=True)

    pcamodel = PCA().fit(y)
    # kcamodel = DCA(T=5, d=1).fit(y)
    kcamodel = KCA(T=5, d=1).fit(y)
    pcacoef[i] = pcamodel.components_.T
    
    variance[i] = pcamodel.explained_variance_ratio_
    
    # Measure PI orthogonal to the projection direction
    pi[i, 0] = kcamodel.score()
    
    v = np.squeeze(kcamodel.coef_)
    v /= np.linalg.norm(v)
    dcacoef[i, :, 0] = v
    u = np.random.normal(size=(2,))
    u -= (v @ u) * v
    u /= np.linalg.norm(u)
    dcacoef[i, :, 1] = u

    pi[i, 1] = calc_pi_from_data(y, proj=u[:, np.newaxis], T=5)
    pca_dca_angle[i] = np.arccos(np.abs(pcamodel.components_[0, :] @ kcamodel.coef_.ravel()))
    

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(30, 10))

start_idx = 2500
end_idx = 5000

for i, idx in enumerate([0,1,3]):
#     ellipse_pca = Ellipse((0, 0), height=2*variance[idx, 0] , width=2*variance[idx, 1], edgecolor='k',
#                           facecolor='r', alpha=0.1, 
#                           angle=180/np.pi * np.arctan2(pcacoef[idx, 0, 0], pcacoef[idx, 1, 0]))
#     ax[i].add_artist(ellipse_pca)
    
#     ax[i].quiver(*np.array([[0, 0], [0, 0]]), variance[idx, 0] * pcacoef[idx, :, 0], 
#                  variance[idx, 1] * pcacoef[idx, :, 1], color=['r','r'],
#                  angles='xy', scale_units='xy', scale=1)

    ellipse_pca = Ellipse((0, 0), height=2*sseig[idx, 0]/np.sum(sseig[idx]), width=2*sseig[idx, 1]/np.sum(sseig[idx]), 
                          edgecolor='none',facecolor='r', alpha=0.1, 
                          angle=180/np.pi * np.arctan2(sscoef[idx, 0, 0], sscoef[idx, 1, 0]))
    ellipse_pca2 = Ellipse((0, 0), height=2*sseig[idx, 0]/np.sum(sseig[idx]), width=2*sseig[idx, 1]/np.sum(sseig[idx]), 
                          edgecolor='r',facecolor='none', linewidth=2, 
                          angle=180/np.pi * np.arctan2(sscoef[idx, 0, 0], sscoef[idx, 1, 0]))

    l1 = ax[i].arrow(0, 0, sseig[idx, 1]/np.sum(sseig[idx]) * sscoef[idx, 1, 0], 
                sseig[idx, 1]/np.sum(sseig[idx]) * sscoef[idx, 0, 0],
                color='r', head_width=0.05, length_includes_head=True)
    ax[i].arrow(0, 0, sseig[idx, 0]/np.sum(sseig[idx]) * sscoef[idx, 1, 1], 
                sseig[idx, 0]/np.sum(sseig[idx]) * sscoef[idx, 0, 1],
                color='r', head_width=0.05, length_includes_head=True)



    ax[i].add_artist(ellipse_pca)
    ax[i].add_artist(ellipse_pca2)

    
    ellipse_dca = Ellipse((0, 0), height=2*pi[idx, 0]/np.sum(pi[idx]), width=2*pi[idx, 1]/np.sum(pi[idx]), 
                          edgecolor='none', alpha=0.1, facecolor='k', 
                          angle=180/np.pi * np.arctan2(dcacoef[idx, 0, 0], dcacoef[idx, 1, 0]))
    ellipse_dca2 = Ellipse((0, 0), height=2*pi[idx, 0]/np.sum(pi[idx]), width=2*pi[idx, 1]/np.sum(pi[idx]), 
                           edgecolor='k', facecolor='none', linewidth=2,
                           angle=180/np.pi * np.arctan2(dcacoef[idx, 0, 0], dcacoef[idx, 1, 0]))

    ax[i].add_artist(ellipse_dca)
    ax[i].add_artist(ellipse_dca2)

    if i == 0:
        l2 = ax[i].arrow(0, 0, pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 1, 0], 
                pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 0, 0],
                color='k', head_width=0.05, length_includes_head=True)

        ax[i].arrow(0, 0, -1*pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 1, 1], 
                    pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 0, 1],
                    color='k', head_width=0.05, length_includes_head=True)
    if i == 1:
        l2 = ax[i].arrow(0, 0, pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 1, 0], 
                pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 0, 0],
                color='k', head_width=0.05, length_includes_head=True)

        ax[i].arrow(0, 0, pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 1, 1], 
                    pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 0, 1],
                    color='k', head_width=0.05, length_includes_head=True)

    if i == 2:
        l2 = ax[i].arrow(0, 0, -1*pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 1, 0], 
                -1*pi[idx, 1]/np.sum(pi[idx]) * dcacoef[idx, 0, 0],
                color='k', head_width=0.05, length_includes_head=True)

        ax[i].arrow(0, 0, -1*pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 1, 1], 
                    -1*pi[idx, 0]/np.sum(pi[idx]) * dcacoef[idx, 0, 1],
                    color='k', head_width=0.05, length_includes_head=True)
        
    ax[i].set_xlim([-1, 1])
    ax[i].set_ylim([-1, 1])

    ax[i].set_xticks([-1, 0, 1])
    ax[i].set_yticks([-1, 0, 1])
    ax[i].tick_params(axis='both', which='major', labelsize=24)

    ax[i].spines['left'].set_position('center')
    ax[i].spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax[i].spines['right'].set_color('none')
    ax[i].spines['top'].set_color('none')
    
    ax[i].set_xlabel('Neuron 2', fontsize=24, labelpad=280)
    ax[i].set_ylabel('Neuron 1', fontsize=24, labelpad=240)

    ax[i].legend([l1, l2], ['PCA', 'DCA'], fontsize=20)
    
    # plt.savefig('hwni/2dim_schematic.pdf', bbox_inches='tight', pad_inches=0)